In [1]:
import pandas as pd

In [2]:
SUBJECT = "Molecular biology"
FILE_PATH = "csv_files/"+ SUBJECT.lower().replace(" ","_") + ".csv"
df = pd.read_csv(FILE_PATH)

In [3]:
df

,CONCEPT,RANK,PARENT,LOCK
0,molecular biology,0,NaN,1
1,chromosome,1,molecular biology,0
2,gene,1,molecular biology,0
3,DNA polymerase,1,molecular biology,0
4,helicase,1,molecular biology,0
...,...,...,...,...
806,Molecular basis,1,molecular biology,0
807,diseases,1,molecular biology,0
808,drug development,1,molecular biology,0
809,technical terms,1,molecular biology,0


In [4]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(temperature=0.1)

subject = "molecular biology"

prompt_template_backwards = PromptTemplate(
            input_variables=["subject","concept"],
            template = 'Give me a list of concepts of {subject} which I need to understand before I can understand "{concept}" the technical terms. Give me only the terms.'
        )

backwards_chain = LLMChain(llm=llm,prompt=prompt_template_backwards,output_key="concept_list")

import re

In [5]:
values = df["CONCEPT"].values[1:]

In [6]:
file_path = "backwards_files/"+ subject.lower().replace(" ","_") + ".csv"
new_df = pd.DataFrame(columns=["CONCEPT","PARENT"])

for concept in values: 
    response = backwards_chain.invoke({"concept":concept,"subject":subject})
    response = response["concept_list"]
    regex_pattern="\s*\n+\d+\.\s*"
    concept_list = re.split(regex_pattern,response)
    concept_list = [item for item in concept_list if item!=""]
    for new_concept in concept_list: 
        new_df.loc[len(new_df)] = {"PARENT":concept,"CONCEPT":new_concept}

In [8]:
new_df

,CONCEPT,PARENT
0,DNA,chromosome
1,Gene,chromosome
2,Nucleotide,chromosome
3,RNA,chromosome
4,Protein,chromosome
...,...,...
23660,CRISPR/Cas9,extract
23661,RNA interference (RNAi),extract
23662,Gene editing,extract
23663,Mutagenesis,extract


In [10]:
new_df.to_csv("backwards_files/molecular_biology.csv",index=False)